# Cats vs Dogs ConvNet (PyTorch)

원본 `.py` 스크립트를 Jupyter Notebook 형태로 재구성했습니다. 각 단계는 독립 셀로 나뉘어 실행 순서에 맞게 배치되어 있습니다.

## 0. 필요한 패키지 설치(필요 시)
로컬 환경에 미설치라면 아래 셀을 실행하세요.

In [ ]:
# 필요 시만 실행
%pip install kagglehub torch torchvision matplotlib --quiet


## 1. 기본 설정 및 임포트

In [ ]:
import os
import shutil
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import kagglehub
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")

## 2. Kaggle 데이터셋 다운로드(선택)

In [ ]:
path = kagglehub.dataset_download("salader/dogs-vs-cats")
print("KaggleHub dataset path:", path)


## 3. 경로 및 하이퍼파라미터

In [ ]:
original_dir = pathlib.Path("07_Deep_Learning/data/cats_and_dogs") / "train"
new_base_dir = pathlib.Path("07_Deep_Learning/data/cats_and_dogs_small")

batch_size = 32
img_height = 180
img_width = 180
num_epochs = 3
model_save_path = "convnet_from_scratch.pth"

print("Path to original dataset:", original_dir)
print("Path to subset dataset:", new_base_dir)

## 4. 데이터셋 서브셋 생성 함수

In [ ]:
def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        # KaggleHub 데이터는 이미 cats/dogs 폴더로 구분되어 있음
        source_dir = original_dir / f"{category}s"  # cats 또는 dogs 폴더
        dest_dir = new_base_dir / subset_name / category
        os.makedirs(dest_dir, exist_ok=True)
        
        # 파일명은 cat.0.jpg, dog.0.jpg 형태
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            src_path = source_dir / fname
            dst_path = dest_dir / fname
            if src_path.exists():  # 파일이 존재하는 경우만 복사
                shutil.copyfile(src=src_path, dst=dst_path)

if not os.path.exists(new_base_dir):
    print("데이터셋 서브셋 생성 중...")
    make_subset("train", start_index=0, end_index=1000)
    make_subset("validation", start_index=1000, end_index=1500)
    make_subset("test", start_index=1500, end_index=2500)
    print("데이터셋 서브셋 생성 완료.")
else:
    print("서브셋 디렉토리가 이미 존재합니다.")

## 5. 데이터 증강 및 전처리

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    transforms.ToTensor()
])

val_test_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor()
])

## 6. 데이터셋 로드 및 DataLoader 생성

In [ ]:
# 데이터셋 경로 확인
for subset in ["train", "validation", "test"]:
    subset_path = new_base_dir / subset
    if not subset_path.exists():
        print(f"경고: {subset_path} 경로가 존재하지 않습니다.")
    else:
        print(f"{subset} 데이터셋 경로: {subset_path}")

train_dataset = datasets.ImageFolder(new_base_dir / "train", transform=train_transforms)
validation_dataset = datasets.ImageFolder(new_base_dir / "validation", transform=val_test_transforms)
test_dataset = datasets.ImageFolder(new_base_dir / "test", transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print(f"데이터셋 크기 - 훈련: {len(train_dataset)}, 검증: {len(validation_dataset)}, 테스트: {len(test_dataset)}")
print(f"클래스: {train_dataset.classes}")

## 7. 모델 정의

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 128, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(128, 256, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(256, 256, kernel_size=3, padding='same'),
            nn.ReLU()
        )
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(256 * 11 * 11, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

model = ConvNet().to(device)
print(model)

## 8. 최적화기와 손실 함수

In [ ]:
optimizer = optim.RMSprop(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

## 9. 모델 훈련

In [ ]:
best_val_loss = float('inf')
history = {'accuracy': [], 'val_accuracy': [], 'loss': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            predicted = (outputs > 0.5).float()
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    epoch_val_loss = val_loss / len(validation_loader.dataset)
    epoch_val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"train_loss: {epoch_loss:.4f}, train_acc: {epoch_acc:.4f} - "
          f"val_loss: {epoch_val_loss:.4f}, val_acc: {epoch_val_acc:.4f}")

    history['loss'].append(epoch_loss)
    history['accuracy'].append(epoch_acc)
    history['val_loss'].append(epoch_val_loss)
    history['val_accuracy'].append(epoch_val_acc)

    if epoch_val_loss < best_val_loss:
        print(f"Validation loss improved from {best_val_loss:.4f} to {epoch_val_loss:.4f}. Saving model...")
        best_val_loss = epoch_val_loss
        torch.save(model.state_dict(), model_save_path)

## 10. 훈련 결과 시각화

In [ ]:
accuracy = history["accuracy"]
val_accuracy = history["val_accuracy"]
loss = history["loss"]
val_loss = history["val_loss"]
epochs = range(1, len(accuracy) + 1)

import matplotlib.pyplot as plt

plt.figure()
plt.plot(epochs, accuracy, label="Training accuracy")
plt.plot(epochs, val_accuracy, label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.show()

plt.figure()
plt.plot(epochs, loss, label="Training loss")
plt.plot(epochs, val_loss, label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

## 11. 최고 성능 모델 로드 및 테스트 평가

In [ ]:
test_model = ConvNet().to(device)
test_model.load_state_dict(torch.load(model_save_path, map_location=device))
test_model.eval()

test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)
        outputs = test_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        predicted = (outputs > 0.5).float()
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = test_correct / test_total
print(f"테스트 정확도: {test_acc:.3f}")